# maxsmi
## Analysis of results

This notebook serves to analyse the results of the simulations ran on the Curta cluster.

## Prerequisites
This notebook will run under the condition that some simulations have been run, e.g.
```
(maxsmi) $ python maxsmi/full_workflow.py --task ESOL --aug-strategy-train augmentation_with_duplication --aug-nb-train 10 --aug-nb-test 10

```

Have a look at the [README](https://github.com/t-kimber/maxsmi/blob/main/README.md) page for more details.

In [ ]:
#  !pip install flake8 pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
from maxsmi.utils_analysis import retrieve_metric
import pandas as pd
from IPython.display import display
import numpy as np
import dataframe_image as dfi

In [3]:
# To show the full pandas data frame with the full grid
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Dataset
We consider the following datasets:

- ESOL
- free_solv
- lipophilicity

In [4]:
# TASK = "lipophilicity"
TASK = "ESOL"
# TASK = "free_solv"

### Grid: augmentation number

The models were run of a fine grid augmentation: from 1 to 20 with a step of 1 as well as a coarser grid: from 20 to 100 with a step of 10.

In [5]:
fine_grid = [elem for elem in range(0, 21, 1)]
coarse_grid = [elem for elem in range(10, 110, 10)]

temp_grid = [elem for elem in range(30, 110, 10)]
full_grid = fine_grid + temp_grid

In [6]:
def array_by_strategy(augmentation_strategy,
                      task="ESOL",
                      set_="test",
                      metric="rmse",
                      grid=full_grid):
    """

    Returns an array of a given metric for a given task.

    Parameters
    ----------
    augmentation_strategy : str
        The augmentation strategy to consider,
        e.g. `augmentation_with_reduced_duplication`.
    task : str, default `ESOL`
        The dataset to consider: ESOL, free_solv or lipophilicity.
    set_ : str, default `test`
        The evaluation set, train or test.
    metric: str, default `rmse`
        The performance metric to consider,
        such as the root mean squared error (rmse).
    grid: list
        The augmentation number to retrieve.

    Returns
    -------
    np.array
        Numeric values for `metric`.
    """

    models = ["CONV1D", "CONV2D", "RNN"]

    result_array = np.zeros((len(grid), len(models)))

    if augmentation_strategy == "augmentation_maximum_estimation":
        if task == "ESOL":
            task = "ESOL_SMALL"
            # Estimated maximum strategy was only evaluated
            # on a subset of ESOL, because time-intensive purposes.

    for i, model in enumerate(models):
        for j, augmentation_num in enumerate(grid):
            try:
                y = retrieve_metric(
                    metric,
                    set_,
                    task,
                    augmentation_strategy,
                    augmentation_num,
                    augmentation_strategy,
                    augmentation_num,
                    model,
                )
            except FileNotFoundError:
                y = np.nan
            result_array[j, i] = y
    return result_array

## Retrieve performance by augmentation strategy

In [7]:
results_without_dupl = array_by_strategy("augmentation_without_duplication",
                                         task=TASK)
results_with_dupl = array_by_strategy("augmentation_with_duplication",
                                      task=TASK)
results_with_red_dupl = array_by_strategy("augmentation_with_reduced_\
duplication",
                                          task=TASK)
results_max_est = array_by_strategy("augmentation_maximum_estimation",
                                    task=TASK)
results_no_aug = array_by_strategy("no_augmentation",
                                   task=TASK)

_Note:_

The augmentation strategy `estimated maximum` was not evaluated on the lipohilicity dataset.

In [8]:
if TASK == "lipophilicity":
    full_results = np.concatenate([results_without_dupl,
                                   results_with_dupl,
                                   results_with_red_dupl,
                                   results_no_aug], axis=1)
    column_name = [
        ('with duplication', 'CONV1D'),
        ('with duplication', 'CONV2D'),
        ('with duplication', 'RNN'),
        ('without duplication', 'CONV1D'),
        ('without duplication', 'CONV2D'),
        ('without duplication', 'RNN'),
        ('with reduced duplication', 'CONV1D'),
        ('with reduced duplication', 'CONV2D'),
        ('with reduced duplication', 'RNN'),
        ('no augmentation', 'CONV1D'),
        ('no augmentation', 'CONV2D'),
        ('no augmentation', 'RNN'),
    ]

else:
    column_name = [
        ('with duplication', 'CONV1D'),
        ('with duplication', 'CONV2D'),
        ('with duplication', 'RNN'),
        ('without duplication', 'CONV1D'),
        ('without duplication', 'CONV2D'),
        ('without duplication', 'RNN'),
        ('with reduced duplication', 'CONV1D'),
        ('with reduced duplication', 'CONV2D'),
        ('with reduced duplication', 'RNN'),
        ('no augmentation', 'CONV1D'),
        ('no augmentation', 'CONV2D'),
        ('no augmentation', 'RNN'),
        ('estimated maximum', 'CONV1D'),
        ('estimated maximum', 'CONV2D'),
        ('estimated maximum', 'RNN'),
    ]
    full_results = np.concatenate([results_without_dupl,
                                   results_with_dupl,
                                   results_with_red_dupl,
                                   results_no_aug,
                                   results_max_est], axis=1)

## Table

Generate the pandas dataframe with results, with multicolumns.

In [9]:
dataframe = pd.DataFrame(full_results,
                         columns=column_name,
                         index=full_grid)

In [10]:
dataframe.columns = pd.MultiIndex.from_tuples(dataframe.columns,
                                              names=['Strategy', 'Model'])
dataframe

Strategy with duplication                     without duplication            \
Model              CONV1D    CONV2D       RNN              CONV1D    CONV2D   
0                     NaN       NaN       NaN                 NaN       NaN   
1                0.963811  1.008730  1.015847            0.974946  0.978425   
2                0.784799  0.787359  0.963559            0.786002  0.768473   
3                0.785119  0.726199  0.896327            0.784462  0.961557   
4                0.732047  0.760507  0.881402            0.718319  0.761477   
5                0.716140  0.748090  0.790791            0.715752  0.723403   
6                0.666425  0.742724  0.788418            0.678882  0.695390   
7                0.660437  0.675508  0.772992            0.667486  0.669962   
8                0.712453  0.692367  0.742918            0.666258  0.699808   
9                0.641594  0.761000  0.727134            0.642232  0.654711   
10               0.646289  0.689373  0.729067            0.662555  0.706197   
11               0.637987  0.667934  0.696173            0.620301  0.719787   
12               0.606338  0.666147  0.715439            0.615335  0.652091   
13               0.621301  0.691624  0.697748            0.613077  0.664250   
14               0.632709  0.656025  0.702251            0.631364  0.645152   
15               0.622017  0.640076  0.676383            0.595682  0.651618   
16               0.624427  0.679592  0.725603            0.615131  0.661971   
17               0.607653  0.674651  0.688851            0.615260  0.660969   
18               0.614797  0.671369  0.704529            0.605668  0.632586   
19               0.604722  0.663998  0.665781            0.604340  0.657952   
20               0.615234  0.653548  0.655797            0.604247  0.648622   
30               0.612172  0.664392  0.625931            0.592476  0.668374   
40               0.582991  0.683591  0.625601            0.594831  0.656024   
50               0.593026  0.688142  0.640793            0.583355  0.661807   
60               0.589162  0.666454  0.616007            0.584351  0.659128   
70               0.587793  0.659678  0.607363            0.577117  0.675444   
80               0.581548  0.647376  0.641597            0.572651  0.674519   
90               0.589152  0.676252  0.599951            0.597684  0.663725   
100              0.579559  0.649524  0.591450            0.596498  0.657759   

Strategy           with reduced duplication                      \
Model          RNN                   CONV1D    CONV2D       RNN   
0              NaN                      NaN       NaN       NaN   
1         1.019943                 0.963551  0.986388  1.022108   
2         0.935091                 0.785418  0.768814  0.943226   
3         0.898622                 0.784557  0.728090  0.805387   
4         0.847465                 0.738864  0.773636  0.817119   
5         0.788511                 0.712385  0.729791  0.792996   
6         0.770905                 0.673136  0.714351  0.760015   
7         0.774945                 0.657508  0.683214  0.763586   
8         0.744323                 0.672433  0.721333  0.732899   
9         0.717657                 0.640726  0.671181  0.714916   
10        0.695956                 0.647010  0.692022  0.752159   
11        0.759613                 0.639272  0.653327  0.695636   
12        0.673407                 0.612839  0.677616  0.670479   
13        0.681923                 0.615132  0.669166  0.720339   
14        0.670333                 0.627518  0.631123  0.686963   
15        0.671563                 0.626178  0.649551  0.681355   
16        0.637770                 0.623892  0.659524  0.670424   
17        0.667098                 0.625988  0.657437  0.657737   
18        0.660224                 0.591740  0.697325  0.744888   
19        0.641645                 0.599431  0.749248  0.682034   
20        0.630336                 0.605080  0.665842  0.631600   
30       

## Cosmetics

We beautify the dataframe for readibily.

In [11]:
def color_nan_white(val):
    """Color the nan text white"""
    if np.isnan(val):
        return 'color: white'

In [12]:
def color_nan_white_background(val):
    """Color the nan cell background white"""
    if np.isnan(val):
        return 'background-color: white'

In [13]:
df_styler = dataframe.style.\
    set_caption(f"Data: {TASK}").\
    format("{:.3f}").\
    background_gradient(cmap='Purples', axis=None).\
    applymap(lambda x: color_nan_white(x)).\
    applymap(lambda x: color_nan_white_background(x)).\
    highlight_min(color="yellow", axis=None)

df_styler

In [14]:
dfi.export(df_styler, f"dataframe_{TASK}.png")